In [309]:
import pandas as pd
from tqdm import tqdm
import numpy as np
from konlpy.tag import Mecab
from gensim.models import Word2Vec
from scipy.spatial import distance
import re
import warnings
warnings.filterwarnings("ignore")

In [310]:
mecab = Mecab()

In [311]:
df = pd.read_csv("./test.csv", header=None)
df.columns = ['sentence']

In [312]:
df = df.iloc[0:9]

In [317]:
for i in range(len(df)):
    df['sentence'][i] = re.sub('[-=+,#/\?:^$©@*\"※~&%｜ㆍ!』\\‘”|\(\)\[\]\<\>`·\'’“…》]', '', df['sentence'][i])
    df['sentence'][i] = df['sentence'][i].split('.')   

In [318]:
result = []
for i in range(len(df)):
    temp = []
    for j in df['sentence'][i]:
        temp.append(mecab.morphs(j))
    result.append(temp)
df['token'] = result

In [319]:
df

,sentence,token
0,"[영국 옥스퍼드대 신종 코로나바이러스 감염증 코로나19 백신 연구, 로이터 뉴스...","[[영국, 옥스퍼드, 대, 신종, 코로나, 바이러스, 감염증, 코로나, 19, 백신..."
1,[크리스탈지노믹스가 수술이 힘든 진행성췌장암 환자에게 아이발티노스타트와 젬시타빈 얼...,"[[크리스탈, 지노, 믹스, 가, 수술, 이, 힘든, 진행, 성, 췌장암, 환자, ..."
2,"[제주시는 최근 신산공원 내 저류지를 개방해 반려동물 자율쉼터를 운영겠다고 했다, ...","[[제주, 시, 는, 최근, 신산, 공원, 내, 저류지, 를, 개방, 해, 반려, ..."
3,"[5월엔 43, 1 서울경제 미국 공급관리협회 ISM 는 1일 현지시간 6월 제조업...","[[5, 월, 엔, 43], [1, 서울경제, 미국, 공급, 관리, 협회, ISM,..."
4,[서울경제 극자외선용 EUV 포토레지스트를 생산하는 일본 기업 TOK가 국내에서 생...,"[[서울경제, 극자외선, 용, EUV, 포토레지스트, 를, 생산, 하, 는, 일본,..."
5,[617 부동산 대책 내용에 반발하는 회원들이 만든 인터넷 카페가 포털 사이트에 항...,"[[617, 부동산, 대책, 내용, 에, 반발, 하, 는, 회원, 들, 이, 만든,..."
6,[한독과 대한약사회가 지난 1일 서울 서초구 대한약사회관에서 제49회 약연상 시상식...,"[[한독, 과, 대한, 약사회, 가, 지난, 1, 일, 서울, 서초구, 대한, 약사..."
7,[요양보호사 광주 46번서 요양시설교회로 전파12명 추가 감염 서울 관악구서만 7명...,"[[요양, 보호사, 광주, 46, 번, 서, 요양, 시설, 교회, 로, 전파, 12..."
8,[아시아경제 뉴욕 백종민 특파원 미국 공급관리협회 ISM 는 1일 현지시간 6월 제...,"[[아시아, 경제, 뉴욕, 백종민, 특파원, 미국, 공급, 관리, 협회, ISM, ..."


In [320]:
#stopwords list화
temp = pd.read_csv("./stop_words.csv",header = None, names = ['a','b'])                                                
stopwords = []
for i in range(temp.shape[0]):
    stopwords.append(temp['a'][i])

In [323]:
for i in range(len(df)):
    for j in df['token'][i]:
        for k in j:
            if k in stopwords:
                j.remove(k)

In [333]:
for i in range(df.shape[0]):
    print(len(df['token'][i]))

7
11
9
10
7
7
5
63
9


In [325]:
# news_tokens = df['tokenized'].tolist()  # data input as list

# params
v_dimension = 300
v_window = 8

# model = Word2Vec(sentences = news_tokens, size = v_dimension, window = v_window, min_count = 5, workers = 4, sg = 0)

In [334]:
df.head()

,sentence,token
0,"[영국 옥스퍼드대 신종 코로나바이러스 감염증 코로나19 백신 연구, 로이터 뉴스...","[[영국, 옥스퍼드, 대, 신종, 코로나, 바이러스, 감염증, 코로나, 19, 백신..."
1,[크리스탈지노믹스가 수술이 힘든 진행성췌장암 환자에게 아이발티노스타트와 젬시타빈 얼...,"[[크리스탈, 지노, 믹스, 수술, 힘든, 진행, 성, 췌장암, 환자, 아이, 발티..."
2,"[제주시는 최근 신산공원 내 저류지를 개방해 반려동물 자율쉼터를 운영겠다고 했다, ...","[[제주, 시, 는, 최근, 신산, 공원, 내, 저류지, 개방, 해, 반려, 동물,..."
3,"[5월엔 43, 1 서울경제 미국 공급관리협회 ISM 는 1일 현지시간 6월 제조업...","[[5, 엔, 43], [1, 서울경제, 미국, 공급, 관리, 협회, ISM, 는,..."
4,[서울경제 극자외선용 EUV 포토레지스트를 생산하는 일본 기업 TOK가 국내에서 생...,"[[서울경제, 극자외선, 용, EUV, 포토레지스트, 생산, 는, 일본, 기업, T..."


In [337]:
def without_normal(tokens):
    result = []
    for token in tokens:
        vectors = []
        for sentence in token:
            init_v = np.array([0.0]*v_dimension)
            count = 0
            for word in sentence:
                word_vectors = model.wv
                if word in word_vectors.vocab:
                    count +=1
                    v = model.wv[word]
                    init_v = init_v + v
            init_v = init_v / count    
            vectors.append(init_v)
        result.append(vectors)
    
    frame = { 'sentence': df['sentence'].tolist(), 'token': df['token'], 'vectors': result }
    result = pd.DataFrame(frame) 
    
    return result

In [338]:
df = without_normal(df['token'])

In [339]:
df

,sentence,token,vectors
0,"[영국 옥스퍼드대 신종 코로나바이러스 감염증 코로나19 백신 연구, 로이터 뉴스...","[[영국, 옥스퍼드, 대, 신종, 코로나, 바이러스, 감염증, 코로나, 19, 백신...","[[-0.10538889992643487, -0.4946817777712237, 0..."
1,[크리스탈지노믹스가 수술이 힘든 진행성췌장암 환자에게 아이발티노스타트와 젬시타빈 얼...,"[[크리스탈, 지노, 믹스, 수술, 힘든, 진행, 성, 췌장암, 환자, 아이, 발티...","[[-0.048106181035295896, 0.06805482770178993, ..."
2,"[제주시는 최근 신산공원 내 저류지를 개방해 반려동물 자율쉼터를 운영겠다고 했다, ...","[[제주, 시, 는, 최근, 신산, 공원, 내, 저류지, 개방, 해, 반려, 동물,...","[[-0.14743310771882534, -0.13834877300541848, ..."
3,"[5월엔 43, 1 서울경제 미국 공급관리협회 ISM 는 1일 현지시간 6월 제조업...","[[5, 엔, 43], [1, 서울경제, 미국, 공급, 관리, 협회, ISM, 는,...","[[0.5259522447983423, -0.27477433408300084, 0...."
4,[서울경제 극자외선용 EUV 포토레지스트를 생산하는 일본 기업 TOK가 국내에서 생...,"[[서울경제, 극자외선, 용, EUV, 포토레지스트, 생산, 는, 일본, 기업, T...","[[0.36697180202121243, -0.017432801754159087, ..."
5,[617 부동산 대책 내용에 반발하는 회원들이 만든 인터넷 카페가 포털 사이트에 항...,"[[617, 부동산, 대책, 내용, 반발, 는, 회원, 이, 만든, 인터넷, 카페,...","[[0.3823689195327461, 0.2790889333467931, 0.17..."
6,[한독과 대한약사회가 지난 1일 서울 서초구 대한약사회관에서 제49회 약연상 시상식...,"[[한독, 대한, 약사회, 지난, 1, 서울, 서초구, 대한, 약사, 회관, 제, ...","[[0.24371248099487275, -0.24728807112842333, 0..."
7,[요양보호사 광주 46번서 요양시설교회로 전파12명 추가 감염 서울 관악구서만 7명...,"[[요양, 보호사, 광주, 46, 번, 서, 요양, 시설, 교회, 전파, 12, 명...","[[-0.09926367482170463, -0.027358140107244255,..."
8,[아시아경제 뉴욕 백종민 특파원 미국 공급관리협회 ISM 는 1일 현지시간 6월 제...,"[[아시아, 경제, 뉴욕, 백종민, 특파원, 미국, 공급, 관리, 협회, ISM, ...","[[0.16775038433843292, 0.15005380259826778, 0...."


In [340]:
model.wv.most_similar("열정")

[('뜨거운', 0.7520753145217896),
 ('용광로', 0.7174553871154785),
 ('든든', 0.7123266458511353),
 ('헌신', 0.7016264200210571),
 ('동반자', 0.6784043312072754),
 ('열심히', 0.657756507396698),
 ('애정', 0.6551125645637512),
 ('되찾', 0.6547534465789795),
 ('귀', 0.6525612473487854),
 ('기울이', 0.6445311307907104)]

In [361]:
df.head(20)

,sentence,token,sentence_embedding,SimMatrix
0,"[영국 옥스퍼드대 신종 코로나바이러스 감염증 코로나19 백신 연구, 로이터 뉴스...","[[영국, 옥스퍼드, 대, 신종, 코로나, 바이러스, 감염증, 코로나, 19, 백신...","[[-0.10538889992643487, -0.4946817777712237, 0...","[[1.0000000000000002, 0.6912867912899513, 0.68..."
1,[크리스탈지노믹스가 수술이 힘든 진행성췌장암 환자에게 아이발티노스타트와 젬시타빈 얼...,"[[크리스탈, 지노, 믹스, 수술, 힘든, 진행, 성, 췌장암, 환자, 아이, 발티...","[[-0.048106181035295896, 0.06805482770178993, ...","[[1.0, 0.6544912768507111, 0.8582937455192681,..."
2,"[제주시는 최근 신산공원 내 저류지를 개방해 반려동물 자율쉼터를 운영겠다고 했다, ...","[[제주, 시, 는, 최근, 신산, 공원, 내, 저류지, 개방, 해, 반려, 동물,...","[[-0.14743310771882534, -0.13834877300541848, ...","[[1.0000000000000002, 0.2773678980064033, 0.85..."
3,"[5월엔 43, 1 서울경제 미국 공급관리협회 ISM 는 1일 현지시간 6월 제조업...","[[5, 엔, 43], [1, 서울경제, 미국, 공급, 관리, 협회, ISM, 는,...","[[0.5259522447983423, -0.27477433408300084, 0....","[[1.0000000000000004, 0.3885150377631291, 0.41..."
4,[서울경제 극자외선용 EUV 포토레지스트를 생산하는 일본 기업 TOK가 국내에서 생...,"[[서울경제, 극자외선, 용, EUV, 포토레지스트, 생산, 는, 일본, 기업, T...","[[0.36697180202121243, -0.017432801754159087, ...","[[1.0000000000000002, 0.6953652923371252, 0.55..."
5,[617 부동산 대책 내용에 반발하는 회원들이 만든 인터넷 카페가 포털 사이트에 항...,"[[617, 부동산, 대책, 내용, 반발, 는, 회원, 이, 만든, 인터넷, 카페,...","[[0.3823689195327461, 0.2790889333467931, 0.17...","[[1.0, 0.6642733424028365, 0.9183629597822554,..."
6,[한독과 대한약사회가 지난 1일 서울 서초구 대한약사회관에서 제49회 약연상 시상식...,"[[한독, 대한, 약사회, 지난, 1, 서울, 서초구, 대한, 약사, 회관, 제, ...","[[0.24371248099487275, -0.24728807112842333, 0...","[[1.0000000000000004, 0.15691616376022074, 0.1..."
7,[요양보호사 광주 46번서 요양시설교회로 전파12명 추가 감염 서울 관악구서만 7명...,"[[요양, 보호사, 광주, 46, 번, 서, 요양, 시설, 교회, 전파, 12, 명...","[[-0.09926367482170463, -0.027358140107244255,...","[[1.0000000000000007, 0.8432958366018695, 0.12..."
8,[아시아경제 뉴욕 백종민 특파원 미국 공급관리협회 ISM 는 1일 현지시간 6월 제...,"[[아시아, 경제, 뉴욕, 백종민, 특파원, 미국, 공급, 관리, 협회, ISM, ...","[[0.16775038433843292, 0.15005380259826778, 0....","[[1.0000000000000002, 0.29893533873087097, 0.2..."


In [342]:
df.rename(columns = {'vectors': 'sentence_embedding'}, inplace=True)

In [354]:
def similarity_matrix(sentence_embedding):
    sim_mat = np.zeros([len(sentence_embedding), len(sentence_embedding)])
    for i in range(len(sentence_embedding)):
        for j in range(len(sentence_embedding)):
            try:
                sim_mat[i][j] = cosine_similarity(sentence_embedding[i].reshape(1, v_dimension),
                                                  sentence_embedding[j].reshape(1, v_dimension))[0,0]
            except:
                print(sentence_embedding[i])
                print(sentence_embedding[j])

    return sim_mat


In [355]:
df['SimMatrix'] = df['sentence_embedding'].apply(similarity_matrix)
df['SimMatrix']

[-1.05388900e-01 -4.94681778e-01  2.46102763e-01  6.32467800e-01
 -3.34022814e-01  4.47936446e-01 -9.51604541e-02  1.96118177e-01
 -4.79102689e-01  4.63354818e-01 -1.65807874e-01  6.01873670e-01
 -2.04913581e-01 -6.61579256e-01  3.10593208e-01 -3.49608358e-01
  8.17687993e-01  3.09642112e-01 -1.37309134e-01 -2.09996235e-01
 -9.02311019e-01 -4.34022921e-01  6.77608675e-02  1.05390588e+00
  5.29239380e-02 -5.47817639e-01  6.15191753e-01  3.45186850e-01
  2.64451460e-01  1.84682091e-01  7.70907010e-01 -2.42486447e-02
 -1.81279488e-01  3.11476688e-01  3.85436938e-01  5.80764521e-01
 -8.22491253e-01  2.57189653e-01 -1.40681062e-01 -1.00283602e+00
 -7.84939905e-02  5.74032423e-01  4.38394302e-02 -5.91686155e-01
 -8.11960720e-01  3.97264656e-01  1.21869507e-01  5.82771957e-02
  2.96508253e-01 -1.52375551e-01 -2.49818826e-02 -1.30362846e-02
  1.73073532e-01 -6.29516861e-01  8.73514885e-01 -3.75970376e-01
 -1.21696465e-01  4.11805543e-01  8.25894028e-02  2.79515861e-01
 -1.04600780e-02 -1.98770

[ 5.74569035e-01  1.49825446e-01  1.52324233e+00  5.70241665e-01
 -2.75530646e-01  1.00275202e+00  1.09739999e-01  2.38458152e-01
 -5.56286583e-01  3.71979425e-02  2.11155852e-01 -1.23222463e+00
 -1.63486699e-01  3.75714531e-01 -3.77686515e-01  3.92015796e-01
 -1.39850003e+00  3.13591838e-01  4.41106370e-01 -2.66127755e-01
  4.85468676e-01  5.39314151e-01 -3.70021013e-01 -9.86577968e-02
  3.00291715e-02 -1.45079459e-01  5.51845089e-01  1.01951373e+00
  3.45584899e-02 -9.61073836e-02 -4.05184488e-01  6.13762145e-01
 -1.51483994e-01 -3.87946924e-01 -2.57012189e-01  5.16444138e-01
  1.05146318e+00  9.88809263e-01 -5.92812320e-01 -6.98210701e-01
  7.12189823e-01  5.89857012e-01  6.94802195e-01  4.47500646e-01
  5.09261688e-02 -4.40732489e-01 -8.41353839e-01  1.62967984e-01
 -3.03717891e-01  2.59312754e-01  5.69201698e-01 -4.86522655e-01
  5.37912183e-01  1.97135881e-01 -2.85894175e-02  1.08149906e-01
  1.54147744e-01  1.43136010e-02  2.23441586e-01  1.70204423e+00
  1.66670531e-02 -3.54834

[ 2.79597231e-01 -8.42618592e-02  1.90362985e-01  5.07398936e-01
 -4.40461113e-01 -1.65919390e-01  4.03285599e-01  3.35791400e-01
  1.42261279e-01 -4.58595701e-02 -1.03048751e+00 -3.03867069e-01
 -2.27820101e-01 -1.31402153e-01  6.87391832e-01  3.42336527e-01
  2.38856637e-01  5.14554083e-02  3.14932027e-01  1.07260346e+00
  4.07809429e-02 -2.16348916e-01  4.95965993e-01  2.18558605e-01
 -9.00394410e-02 -2.22338017e-02  4.36667158e-01  3.55218439e-01
  7.50183374e-02 -1.31988487e-01 -9.84377280e-02  1.55639768e-01
  1.12522604e-01  2.66125056e-01  4.28386964e-01  6.28475076e-01
  1.25457730e-01  2.54391953e-01  2.88974908e-01 -4.46057013e-01
 -2.36644864e-02  3.69988064e-01  7.29802178e-01 -2.30747741e-01
 -5.41960565e-01 -1.86408033e-01  2.32709925e-01 -1.88716586e-01
  4.61444819e-01  7.10700750e-02 -3.25180921e-01 -3.72778258e-01
  4.76790953e-01  5.34025365e-01 -1.00761369e-01  2.27541319e-01
 -2.22189975e-01  7.98669150e-01  5.51111742e-01 -5.38045843e-01
 -1.49837101e-01 -4.05269

[-0.03853374  0.28136302 -0.09230814  0.35209312  0.34795529 -0.33361515
 -0.23691213 -0.1938439  -0.47026926 -0.19033168 -0.22547746 -0.61719903
 -1.04683386 -0.41324038  0.41914235 -0.08188416 -0.31969191 -0.18113972
 -0.00980931  0.53384331 -0.01147057  0.31842241  0.06425401 -0.5618224
  0.22317383  0.63208204  0.04566517 -0.63089789 -0.67674134  0.24744242
  0.39366716 -0.08884775 -0.11876695  0.21956107  0.35604587 -1.06873374
 -0.1418603  -0.34621177 -0.2604152   0.19131067 -0.17854247  0.22589052
  0.85499553  0.28688038 -0.28854272 -0.30973891  0.26067135 -0.1057793
  0.15183182  0.75162911 -0.56290812 -0.47948869 -0.07674659  0.37392461
 -0.23520897  0.10419864  0.01976412 -0.11862316 -0.47104702  0.32783329
  0.53191422 -0.18375179 -0.3725364   0.04405081 -0.09810049 -0.57232065
  0.57069066 -0.15241373  0.06047962 -0.42825115  0.28772896  0.26043791
  0.52749289  0.16542737 -0.21037894  0.01838165  0.16928414 -0.05632239
 -0.16032255  0.57135348  0.07332397 -0.4095017   0.4

[-0.11255298 -0.32631938  0.4718157   0.41493984 -0.03630031 -0.36318635
  0.68153667  0.41135198  0.29775412  0.11158817 -0.25524246 -0.29211823
 -0.88341015 -0.47031975  0.38020227 -0.04373726  0.24191637 -0.64953777
  0.1200105   0.30706423 -0.24078282  0.59490507 -0.3278626  -0.13192665
 -0.6033373   0.6157776   0.31407927 -0.08878315  0.19452282  0.24109129
  0.06733879  0.1773026  -0.00809068  0.13404351  0.098516   -0.24059051
 -0.20279528 -0.12609628 -0.45292062  0.02773143 -0.26698792  0.80361178
  0.4325938   0.52800171 -0.83667533 -0.03336983  0.29181076  0.07634293
  0.24749612  0.0417347  -0.38589813 -0.19766206  0.21492147 -0.51231178
  0.45338815 -0.07484323 -0.03187486 -0.04649194 -0.05263459  0.20198187
  0.14751885 -0.30685115 -0.67772527 -0.36176002 -0.32120325 -0.05316598
  0.55701713 -0.38743551  0.24320388 -0.58928669 -0.29867679  0.14749916
  0.57616873  0.10606834 -0.01268172 -0.39776502 -0.15724315 -0.33320603
 -0.72032924  0.11184518  0.0358271   0.30772556  0

...

[-0.00620546 -0.15312221  0.43272481  0.19242275 -0.13732501 -0.13817886
  0.35365783  0.15721877  0.05219604  0.15072267 -0.30678161 -0.1734971
 -0.46493425 -0.31359375  0.287168    0.13729132  0.00314417 -0.21722494
  0.07250251  0.01314641 -0.17342852  0.12411478 -0.31816969 -0.01366172
 -0.21180469  0.15065051  0.00491786 -0.14844945  0.37884904 -0.34337573
  0.24230345  0.27342953  0.16645803  0.06280737  0.01544882 -0.20177674
 -0.26117252 -0.09482553 -0.20080917 -0.18238216 -0.27357096  0.52142914
  0.31177617  0.06467137 -0.11203388  0.18212135  0.05382595  0.1506419
  0.57591249  0.08877386 -0.47854763 -0.18524573  0.087917   -0.335216
  0.27800329  0.10865361 -0.22604747 -0.27187497 -0.18064734  0.35111585
  0.49825812 -0.31421421 -0.29580627 -0.10000299 -0.25143375  0.17583857
  0.45564342  0.20760907 -0.14328218 -0.4233658   0.06138716 -0.2419099
  0.50022017  0.39173418 -0.09679404 -0.37079644  0.15391794 -0.1551505
 -0.52648565 -0.16933913 -0.10494527 -0.09481256  0.02941

[ 0.12327827 -0.09161291  0.23616643  0.1823583   0.1427304  -0.29525153
  0.36696448 -0.15820151  0.07032441 -0.25666816 -0.06190511 -0.45797084
 -0.56382482  0.29129647 -0.02789776 -0.01654838 -0.25370383  0.19204873
  0.40738327 -0.41903922  0.04536297  0.24481134  0.17982589  0.50874701
 -0.25774307  0.03247342  0.36710918  0.37393205  0.17746349 -0.49571924
  0.27589316 -0.0568541   0.06182592 -0.34432212  0.27742434  0.25757445
 -0.08316567  0.27690534 -0.27364351 -0.17921135 -0.12877182  0.21361119
  0.41005851  0.15799109 -0.16134695  0.1670713   0.07313592  0.23770557
  0.30897028  0.09821264 -0.45547163 -0.09362299 -0.10400915  0.03268834
  0.34674656  0.31467265 -0.33497862  0.09224537 -0.13291714  0.51998397
  0.28755508 -0.17961868 -0.06026546 -0.32613196 -0.15813149 -0.13728742
  0.71978576  0.32854274 -0.03778904 -0.04986982  0.2531357  -0.41556482
  0.50819631  0.15637886 -0.20887657 -0.15775695  0.49313975  0.06281391
 -0.43184222 -0.49166677  0.0036474  -0.29263843 -0

[-4.25634517e-01 -1.86966961e-01  2.19187436e-02  2.27041388e-01
  1.71971661e-01  9.30899686e-03 -3.24433459e-02  1.87562057e-02
  1.77103511e-01 -3.70833908e-01  4.45511433e-02 -6.47130680e-02
 -2.02926298e-01  3.87668371e-01  3.24410038e-01 -2.36652981e-02
  1.98022322e-01 -5.26220528e-02  2.91384624e-01  1.52787048e-01
 -2.47660259e-02 -1.02286135e-01  1.97037334e-01  1.88127752e-01
 -8.98249266e-02  1.87902495e-01  1.82519041e-01  8.61286006e-02
  2.82486133e-01 -2.27791755e-01  4.18765120e-01  2.70463981e-01
 -9.44057415e-02 -8.76045098e-02  6.85009297e-01  1.41393567e-01
  1.67235512e-01  7.05681815e-02 -3.72463605e-01 -2.47835160e-01
 -4.54730057e-01  5.35517226e-01  4.19418024e-01  6.17252995e-02
 -1.75081554e-01  3.49762840e-01 -9.55902018e-02  3.41294511e-01
  4.02011749e-01  1.89187516e-01 -4.44970501e-01  1.21677331e-01
  3.20969965e-01  3.24773651e-01  2.24828108e-01  2.86734068e-01
 -3.49722700e-01 -1.62349582e-01 -4.21952078e-01  1.20677715e-01
  6.77107356e-01 -2.12312

[ 2.79597231e-01 -8.42618592e-02  1.90362985e-01  5.07398936e-01
 -4.40461113e-01 -1.65919390e-01  4.03285599e-01  3.35791400e-01
  1.42261279e-01 -4.58595701e-02 -1.03048751e+00 -3.03867069e-01
 -2.27820101e-01 -1.31402153e-01  6.87391832e-01  3.42336527e-01
  2.38856637e-01  5.14554083e-02  3.14932027e-01  1.07260346e+00
  4.07809429e-02 -2.16348916e-01  4.95965993e-01  2.18558605e-01
 -9.00394410e-02 -2.22338017e-02  4.36667158e-01  3.55218439e-01
  7.50183374e-02 -1.31988487e-01 -9.84377280e-02  1.55639768e-01
  1.12522604e-01  2.66125056e-01  4.28386964e-01  6.28475076e-01
  1.25457730e-01  2.54391953e-01  2.88974908e-01 -4.46057013e-01
 -2.36644864e-02  3.69988064e-01  7.29802178e-01 -2.30747741e-01
 -5.41960565e-01 -1.86408033e-01  2.32709925e-01 -1.88716586e-01
  4.61444819e-01  7.10700750e-02 -3.25180921e-01 -3.72778258e-01
  4.76790953e-01  5.34025365e-01 -1.00761369e-01  2.27541319e-01
 -2.22189975e-01  7.98669150e-01  5.51111742e-01 -5.38045843e-01
 -1.49837101e-01 -4.05269

0    [[1.0000000000000002, 0.6912867912899513, 0.68...
1    [[1.0, 0.6544912768507111, 0.8582937455192681,...
2    [[1.0000000000000002, 0.2773678980064033, 0.85...
3    [[1.0000000000000004, 0.3885150377631291, 0.41...
4    [[1.0000000000000002, 0.6953652923371252, 0.55...
5    [[1.0, 0.6642733424028365, 0.9183629597822554,...
6    [[1.0000000000000004, 0.15691616376022074, 0.1...
7    [[1.0000000000000007, 0.8432958366018695, 0.12...
8    [[1.0000000000000002, 0.29893533873087097, 0.2...
Name: SimMatrix, dtype: object

KeyError: 'vectors'